In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from scipy import stats
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')
import matplotlib.cm as cm
import helper

In [2]:
colors = ["#FF0B04", "#F1BE48",
           "#B9975B", "#8B5B29",
           "#524727",
         ]
sns.set_palette(sns.color_palette(colors))

In [3]:
df = pd.read_csv('Ames_Housing_Price_Data.csv', 
                             index_col=0,low_memory = False)

In [4]:
train, test = helper.data_processing_wrapper(df,
                                               num_to_cat_list = ['MSSubClass','MoSold'],
                                             remove_PID = False
                                        )

In [5]:
train['LogSalePrice'] = np.log(train['SalePrice'])
test['LogSalePrice'] = np.log(test['SalePrice'])

In [6]:
categorical = train.select_dtypes(['object','bool']).columns.to_list()

In [7]:
nhds = train.groupby('Neighborhood').median()[['LogSalePrice']]
nhds['LogSalePrice'] = stats.zscore(nhds['LogSalePrice'])

In [8]:
def segment(y):
    if round(y,2) < -0.75:
        return 0
    elif (round(y,2) >= -0.75) and (round(y,2) < 0.75):
        return 1
    else:
        return 2

In [9]:
nhds['Segment'] = nhds.apply(lambda x: segment(x['LogSalePrice']),axis = 1)
nhds.sort_values('LogSalePrice')
seg_dict = nhds.drop('LogSalePrice', axis=1).to_dict()['Segment']

In [10]:
train['Segment'] = train.apply(lambda x: seg_dict[x['Neighborhood']], axis=1)
test['Segment'] = test.apply(lambda x: seg_dict[x['Neighborhood']], axis=1)

In [11]:
X0_train = train.loc[train['Segment']==0,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'Segment'], axis=1)
y0_train = train.loc[train['Segment']==0, 'LogSalePrice']
X0_test = test.loc[test['Segment']==0,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'Segment'], axis=1)
y0_test = test.loc[test['Segment']==0, 'LogSalePrice']
X1_train = train.loc[train['Segment']==1,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'Segment'], axis=1)
y1_train = train.loc[train['Segment']==1, 'LogSalePrice']
X1_test = test.loc[test['Segment']==1,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'Segment'], axis=1)
y1_test = test.loc[test['Segment']==1, 'LogSalePrice']
X2_train = train.loc[train['Segment']==2,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'Segment'], axis=1)
y2_train = train.loc[train['Segment']==2, 'LogSalePrice']
X2_test = test.loc[test['Segment']==2,:].drop(['SalePrice', 'LogSalePrice', 'PID', 'Segment'], axis=1)
y2_test = test.loc[test['Segment']==2, 'LogSalePrice']

In [16]:
pipe = Pipeline(
        [
            ('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown = 'ignore'), categorical)], 
                                                remainder='passthrough')),
            ('scaler', StandardScaler(with_mean=False))
        ]
    )
    
X = pipe.fit_transform(X0_train)
y = y0_train

selector = SelectFromModel(Lasso(alpha=0.05, max_iter=5000))
selector.fit(X,y)
mask = selector.get_support()
feat_names = pipe.named_steps['transformer'].get_feature_names()
names0 = [name for name, boo in zip(feat_names, mask) if boo]
names0

['Cat__x22_N',
 'GrLivArea',
 'OverallQual',
 'OverallCond',
 'YearRemodAdd',
 'BsmtFinSF1',
 'TotalBsmtSF',
 '1stFlrSF',
 'KitchenQual',
 'Fireplaces',
 'GarageArea']

In [20]:
pipe = Pipeline(
        [
            ('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown = 'ignore'), categorical)], 
                                                remainder='passthrough')),
            ('scaler', StandardScaler(with_mean=False))
        ]
    )
    
X = pipe.fit_transform(X1_train)
y = y1_train

selector = SelectFromModel(Lasso(alpha=0.03, max_iter=5000))
selector.fit(X,y)
mask = selector.get_support()
feat_names = pipe.named_steps['transformer'].get_feature_names()
names1 = [name for name, boo in zip(feat_names, mask) if boo]
names1

['Cat__x22_N',
 'GrLivArea',
 'LotArea',
 'OverallQual',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtQual',
 'BsmtFinSF1',
 'TotalBsmtSF',
 'KitchenQual',
 'Fireplaces',
 'GarageCars',
 'GarageArea']

In [21]:
pipe = Pipeline(
        [
            ('transformer', ColumnTransformer([("Cat", OneHotEncoder(handle_unknown = 'ignore'), categorical)], 
                                                remainder='passthrough')),
            ('scaler', StandardScaler(with_mean=False))
        ]
    )
    
X = pipe.fit_transform(X2_train)
y = y2_train

selector = SelectFromModel(Lasso(alpha=0.03, max_iter=5000))
selector.fit(X,y)
mask = selector.get_support()
feat_names = pipe.named_steps['transformer'].get_feature_names()
names2 = [name for name, boo in zip(feat_names, mask) if boo]
names2

['Cat__x11_1Fam',
 'Cat__x11_Twnhs',
 'Cat__x29_New',
 'GrLivArea',
 'OverallQual',
 'ExterQual',
 'BsmtExposure',
 'BsmtFinSF1',
 'TotalBsmtSF',
 '1stFlrSF',
 'KitchenQual',
 'FireplaceQu',
 'GarageCars',
 'GarageArea']